# Voting power - new distribution

In [1]:
import numpy as np
import pandas as pd
import power_index_calculator as calculator

from itertools import permutations
from itertools import product
from itertools import combinations_with_replacement
from tqdm import tqdm

In [2]:
base_line = sorted(tuple([2,2,4,4,6,12,32,8,10,14,10,4,8,2,2]))

In [3]:
std_baseline = np.std(base_line)
print('Baseline standard deviation: {}'.format(std_baseline))

Baseline standard deviation: 7.447594690010102


In [4]:
weights_combinations = []
for digits in tqdm(combinations_with_replacement(list(range(16)), 15)):
    if sum(map(int, digits)) == 35:
        weights_combinations.append(digits)

155117520it [05:56, 435571.43it/s]


In [5]:
print('{} games found'.format(len(weights_combinations)))

10745 games found


### 1st constraints
The order of the regions according to the number of seats must be maintained.

All the regions that currently share the same number of seats must continue having the same number of seats.

In [6]:
filter_weights = []

for w in weights_combinations:
    if ((w[0] == w[1] == w[2] == w[3]) and (w[4] == w[5] == w[6])
        and (w[8] == w[9]) and (w[10]==w[11])):
        filter_weights.append(w)

In [7]:
print('Number of generate weights: {} '.format(len(filter_weights)))

Number of generate weights: 1194 


### Working with power indices

In [8]:
import power_index_calculator as pw

In [9]:
game_2017 = [3,3,5,5,7,16,47,9,11,18,11,5,9,3,3]
quota_2017 = 89

In [10]:
banzhaf_2017 = list(pw.banzhaf(game_2017,quota_2017))
shapley_2017 = list(pw.shapley(game_2017,quota_2017))
deegan_packel_2017 = list(pw.deegan_packel(game_2017,quota_2017))

print('Banzhaf: {}'.format(banzhaf_2017))
print('Shapley: {}'.format(shapley_2017))
print('Deegan Packel: {}'.format(deegan_packel_2017))

Banzhaf: [0.017, 0.017, 0.028, 0.028, 0.039, 0.089, 0.396, 0.051, 0.062, 0.1, 0.062, 0.028, 0.051, 0.017, 0.017]
Shapley: [0.016, 0.016, 0.028, 0.028, 0.039, 0.091, 0.393, 0.051, 0.062, 0.103, 0.062, 0.028, 0.051, 0.016, 0.016]
Deegan Packel: [0.06, 0.06, 0.064, 0.064, 0.064, 0.059, 0.131, 0.065, 0.065, 0.056, 0.065, 0.064, 0.065, 0.06, 0.06]


In [11]:
std_banzhaf_2017 = np.std(banzhaf_2017)
std_shapley_2017 = np.std(shapley_2017)
std_deegan_packel_2017 = np.std(deegan_packel_2017)

print('STD Banzhaf 2017: {}'.format(std_banzhaf_2017))
print('STD Shapley 2017: {}'.format(std_shapley_2017))
print('STD Deegan Packel 2017: {}'.format(std_deegan_packel_2017))

STD Banzhaf 2017: 0.09155049608458347
STD Shapley 2017: 0.09108945542097005
STD Deegan Packel 2017: 0.017375077937475086


### 2st constraint
The standard deviations for the three power indices must be smaller than the corresponding ones obtained for the Γ6 game.

In [12]:
new_games  = []

for w_add in tqdm(filter_weights):
    
    to_add = tuple(map(int, w_add))
    new_weights = tuple(map(sum,zip(to_add,base_line)))
    
    new_banzhaf_std = np.std(list(pw.banzhaf(new_weights,quota_2017)))
    new_shapley_std = np.std(list(pw.shapley(new_weights,quota_2017)))
    new_deegan_packel_std = np.std(list(pw.deegan_packel(new_weights,quota_2017)))
    
    if ((new_banzhaf_std < std_banzhaf_2017)
        and (new_shapley_std < std_shapley_2017)
        and (new_deegan_packel_std < std_deegan_packel_2017)):
            
        new_games.append(new_weights)

100%|██████████| 1194/1194 [18:20<00:00,  1.20it/s]


In [13]:
print('{} games found'.format(len(new_games)))

1083 games found


### 3st constraint
The power indices for each region must be directly proportional to its weight.

In [14]:
new_games_proportional  = []

for new_game in tqdm(new_games):
    
    raw_banzhaf = list(pw.banzhaf(new_game,quota_2017))
    raw_shapley = list(pw.shapley(new_game,quota_2017))
    raw_deegan_packel = list(pw.deegan_packel(new_game,quota_2017))
    
    new_banzhaf_std = np.std(list(pw.banzhaf(new_game,quota_2017)))
    new_shapley_std = np.std(list(pw.shapley(new_game,quota_2017)))
    new_deegan_packel_std = np.std(list(pw.deegan_packel(new_game,quota_2017)))
    
    if ((sorted(raw_banzhaf) == raw_banzhaf) 
        and (sorted(raw_shapley) == raw_shapley)
        and (sorted(raw_deegan_packel) == raw_deegan_packel)):
        
        new_games_proportional.append({'game':new_game,
                                       'std_bz':new_banzhaf_std, 
                                       'std_ss':new_shapley_std,
                                       'std_dp':new_deegan_packel_std})

100%|██████████| 1083/1083 [35:10<00:00,  1.82s/it]


In [15]:
print('{} games found'.format(len(new_games_proportional)))

48 games found


In [17]:
df = pd.DataFrame(new_games_proportional)

In [18]:
df

,game,std_bz,std_dp,std_ss
0,"(2, 2, 2, 2, 4, 4, 4, 6, 9, 9, 14, 14, 19, 23,...",0.076267,0.012593,0.077934
1,"(2, 2, 2, 2, 4, 4, 4, 6, 9, 9, 16, 16, 18, 21,...",0.074456,0.012110,0.075733
2,"(2, 2, 2, 2, 4, 4, 4, 6, 10, 10, 13, 13, 19, 2...",0.080769,0.012960,0.081795
3,"(2, 2, 2, 2, 4, 4, 4, 6, 10, 10, 14, 14, 17, 2...",0.075496,0.012273,0.077540
4,"(2, 2, 2, 2, 4, 4, 4, 6, 11, 11, 13, 13, 18, 2...",0.075456,0.012043,0.077958
5,"(2, 2, 2, 2, 4, 4, 4, 7, 9, 9, 14, 14, 19, 22,...",0.076266,0.012228,0.077697
6,"(2, 2, 2, 2, 4, 4, 4, 7, 9, 9, 15, 15, 18, 21,...",0.076426,0.012629,0.077589
7,"(2, 2, 2, 2, 4, 4, 4, 7, 10, 10, 13, 13, 18, 2...",0.075778,0.012062,0.077469
8,"(2, 2, 2, 2, 4, 4, 4, 7, 10, 10, 16, 16, 18, 2...",0.068901,0.011442,0.070676
9,"(2, 2, 2, 2, 4, 4, 4, 7, 11, 11, 13, 13, 16, 2...",0.080378,0.014263,0.081674
